# Code for BEA Economic Model 

Author: Christopher Blackburn
Last Update: 08/26/20

## Introduction

This is a technical document that describes the data and method for estimating the BEA economic model. The model is based off of Huo, Levchenko, and Pandalai-Nayar's (2020) "International Comovement in the Global Production Network." For the rest of the document, we will refer to this paper as HLP. As we proceed with the document, we highlight the components of the model and the data that goes along with these various components. 

The technical document proceeds as follows. The first section highlights the consumption side of the economy. We present the economic environment as given in HLP and the corresponding data requirements. For the initial version of this document, in which there will more than likely be several iterations, we start by presenting the data used as given in HLP.

The next section goes into the production side of the economy. 

## Households

In the model, each country $n$ is populated with a representative household. The represenative household consumes final goods and supplies labor and capital to industries. The household's problem is given by

$$\max_{\mathcal{F}_n,\lbrace H_{nj} \rbrace} \mathcal{F}_{n} - \sum_{j}H_{nj}^{1+ \frac{1}{\psi}}$$

subject to 
$$P_{n}\mathcal{F}_n = \sum_{j}W_{nj}H_{nj} + R_{nj}K_{nj}$$

The consumption index $\mathcal{F}_{n}$ is defined  assumed to be constant elasticity of substition given by

$$\mathcal{F}_{n}= \left[\sum_j \sum_m \vartheta^{\frac{1}{\rho}}_{mj,n} \mathcal{F}_{mj,n}^{\frac{\rho-1}{\rho}} \right]^{\frac{\rho}{\rho-1}}$$

It should be noted that we deviate from the notation in HLP, and adopt a certain convention throughout the rest of the paper. That is, we will tend to reserve the first index for the <i>source</i> country(-sector) and the second index for the <i>destination</i> country(-sector). When we turn to the matrix notation, we will be explicit about the entries of each matrix. 

We now turn to the first major result of the paper that requires calibration. That is, the representative household in $n$'s expenditure share on final goods $j$ sourced from country $m$ is given by

$$\pi_{mj,n} = \frac{\vartheta_{mj,n}P_{mj,n}^{1-\rho}}{\sum_{k,l}\vartheta_{kl,n}P_{kl,n}^{1-\rho}}$$

### Calibrating Household Expenditure Shares

The first major component of the model that requires calibration is the expenditure share of the representative households. According to HLP, the dataset used to calibrate the final use parameters is the WIOD. The following code segment calibrates these parameters using the WIOD. According to the download portion of the website, the input-ouput tables are in current prices, denoted in millions of dollars. The database covers 28 EU countries and 15 other major countries in the world for the period 2000-2014. However, for the purpose of this exercise we only need to recover the 2014 input-output table. As a check on our code, there are 43 countries (and ROW component) in the data and each country has 56 sectors. This implies the country-sector vector length is $NJ = 44 \times 56 = 2,464$. 

In essence, the following code performs these operations. In the WIOD data, the expenditure by the representative household $n$ on country-sector $mj$ is given by the $NJ \times N$ vector $\mathbf{F}$. To compute expenditure shares, we first compute total household expenditure in $N$ to compute this we use 

$$\mathbf{F}_{total} = \mathbf{\iota}\mathbf{F}$$

where $\mathbf{\iota}$ is a $1 \times NJ$ vector of 1s. Hence, the matrix $\mathbf{F}_{total}$ is a $1 \times N$ matrix where each entry corresponds to each country's household expenditures on final goods. The next step is to compute the final use expenditure shares using the following expression

$$\mathbf{\Pi}^{f} = \left[\mathbf{F}diag(\mathbf{F}_{total})^{-1}\right]^{'}$$

We note that this procedure is general for the $N$ country case. However, we make some adjustments because we role up these expenditures into a rest of the world component. 

In [90]:
'''Download the 2014 WIOT from the WIOD Website'''

import pandas as pd 

# Download the 2014 WIOT from the WIOD website
wiod_2014 = pd.read_excel('http://www.wiod.org/protected3/data16/wiot_ROW/WIOT2014_Nov16_ROW.xlsb',header=2,engine='pyxlsb')
print('WIOT for 2014 Successfully downloaded!')

''' There are some special requirements here. 
    First, the excel file is actually a binary
    excel file. Pandas can read binary excel files,
    but you need to download the pyxlsb package. 
    
    To download use,
    
    pip3 install pyxlsb
    
    You may also need to upgrade pandas,
    
    pip3 install pandas --upgrade
    
    '''

The next segment of code primarily extract the appropriate data components from the WIOT downloaded in the previous code snippet. The following code (generally) proceeds as follows:

1. Extract household spending components by each region
2. Role up expenditures for ROW (sum the columns)
3. Sum the row elements to make sure the matrix is NJ x N, where N=3 and J=56
4. Extract the matrix elements and compute the final expenditure share matrix

In [147]:
'''
--------------------------
Get the relevant data
--------------------------
'''

pd.options.mode.chained_assignment = None 

# Extract the columns with the Chinese data
chn_cols = wiod_2014.columns[(wiod_2014=='CHN').any(axis=0)]

# Extract the columns with the US data
us_cols = wiod_2014.columns[(wiod_2014=='USA').any(axis=0)]

# Extract all others columns 
row_cols=[]
for col_name in wiod_2014.columns:
    if col_name in list(chn_cols)[1:] or col_name in list(us_cols)[1:]:
        pass
    else:
        row_cols.append(col_name)
    
# Rename the first 4 columns
wiod_2014.rename(columns={wiod_2014.columns[0]: 'Industry Code'}, inplace = True)
wiod_2014.rename(columns={wiod_2014.columns[1]: 'Industry Description'}, inplace = True)
wiod_2014.rename(columns={wiod_2014.columns[2]: 'Country'}, inplace = True)
wiod_2014.rename(columns={wiod_2014.columns[3]: 'Industry WIOD Code'}, inplace = True)
        
# Extract the data for each country
chn_cols_list = list(chn_cols)[1:]
chn_cols_list.extend(['Industry Code','Country'])
wiod_chn2014 = wiod_2014[chn_cols_list]
wiod_chn2014 = wiod_chn2014[wiod_chn2014['Country'] != 'TOT'].drop([0,1,2])

us_cols_list = list(us_cols)[1:]
us_cols_list.extend(['Industry Code','Country'])
wiod_usa2014 = wiod_2014[us_cols_list]
wiod_usa2014 = wiod_usa2014[wiod_usa2014['Country'] != 'TOT'].drop([0,1,2])

row_cols_list = list(row_cols)[4:-1]
row_cols_list.extend(['Industry Code','Country'])
wiod_row2014 = wiod_2014[row_cols_list]
wiod_row2014 = wiod_row2014[wiod_row2014['Country'] != 'TOT'].drop([0,1,2])


# Create a function to extract the appropriate columns (CONS_h)
def get_CONSH(col_list):
    return [col for col in col_list if 'CONS_h' in col]

# Get HH spending for the different countries
chn_HH = get_CONSH(wiod_chn2014.columns)
chn_HH.extend(['Industry Code','Country'])
usa_HH = get_CONSH(wiod_usa2014.columns)
usa_HH.extend(['Industry Code','Country'])
row_HH = get_CONSH(wiod_row2014.columns)
row_HH.extend(['Industry Code','Country'])
    
# Extract HH spending from each dataset
chn_HH2014 = wiod_chn2014[chn_HH]
usa_HH2014 = wiod_usa2014[usa_HH]
row_HH2014 = wiod_row2014[row_HH]

# Compute total HOUSEHOLD spending in ROW
row_HH2014['CONS_h.ROW'] = row_HH2014[get_CONSH(wiod_row2014)].sum(axis=1)
row_HH2014 = row_HH2014[['Industry Code','Country','CONS_h.ROW']]

# Compute total HOUSEHOLD spending in CHN
chn_HH2014.columns = ['CONS_h.CHN','Industry Code','Country']

# Compute total HOUSEHOLD spending in USA
usa_HH2014.columns = ['CONS_h.USA','Industry Code','Country']

# Create a function that maps countries into ROW or not
def row_map(x):
    if x == 'USA' or x=='CHN':
        return x
    else:
        return 'ROW'
    
# Collapse the data into spending by region-sector (Regions = CHN,USA,ROW)
chn_HH2014['Source Region'] = chn_HH2014['Country'].apply(lambda x: row_map(x))
chn_HH2014 = chn_HH2014[['Source Region','Industry Code','CONS_h.CHN']].groupby(['Source Region','Industry Code']).sum().reset_index()

usa_HH2014['Source Region'] = usa_HH2014['Country'].apply(lambda x: row_map(x))
usa_HH2014 = usa_HH2014[['Source Region','Industry Code','CONS_h.USA']].groupby(['Source Region','Industry Code']).sum().reset_index()

row_HH2014['Source Region'] = row_HH2014['Country'].apply(lambda x: row_map(x))
row_HH2014 = row_HH2014[['Source Region','Industry Code','CONS_h.ROW']].groupby(['Source Region','Industry Code']).sum().reset_index()


# Merge the datasets together
wiod_HH2014 = pd.merge(chn_HH2014,row_HH2014 ,on=['Source Region','Industry Code'],how='inner')
wiod_HH2014 = pd.merge(wiod_HH2014,usa_HH2014,on=['Source Region','Industry Code'],how='inner')

'''
----------------------------------------------------
Compute the expenditure share matrix
----------------------------------------------------
'''

import numpy as np
from numpy.linalg import inv

# Extract the final expenditure columns as a matrix
F = wiod_HH2014.iloc[:,2:].values

# Create a vector of ones of the same dimension
i = np.ones((1,F.shape[0]))

# Compute total HH expenditures in each region
F_total = np.dot(i,F)

# Diagonalize and invert the matrix
Pi_f = np.transpose(np.dot(F,inv(np.diag(F_total[0]))))

print('Successfully computed the expenditure share matrix!')

Successfully computed the expenditure share matrix!


## Firms

With the household side of the economy calibrated, we turn our attention to the production side of the economy. Each industry $nj$ is populated by a representative firm with access to CRS production function given by

$$Y_{nj} = \left(K_{nj}^{\alpha_j}H_{nj}^{1-\alpha_j}\right)^{\eta_j}X_{nj}^{1-\eta_j}$$

where intermediate input usage is a composite of intermediates inputs sourced from other industries. The composite intermediate is given by

$$X_{nj} = \left(\sum_{i} \sum_{m}\mu_{mi,nj}^{\frac{1}{\epsilon}}X_{mi,nj}^{\frac{\epsilon-1}{\epsilon}}\right)^{\frac{\epsilon}{\epsilon-1}}$$

One important assumption that might be relevant for future interations is that an input's price is given by

$$P_{mi,nj} = \tau_{mi,n}P_{mi}$$